In [ ]:
# Install meridian: from PyPI @ latest release
!pip install --upgrade google-meridian[colab]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 721.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 79.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.7/435.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 933.6/933.6 kB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.3/233.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arviz/__init__.py:39: FutureWarning: 
ArviZ is undergoing a major refactor to improve flexibility and extensibility while maintaining a user-friendly interface.
Some upcoming changes may be backward incompatible.
For details and migration guidance, visit: https://python.arviz.org/en/latest/user_guide/migration_guide.html
  warn(


Your runtime has 50.5 gigabytes of available RAM

Num GPUs Available:  0
Num CPUs Available:  1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

fixed_path = "/content/drive/MyDrive/NLB HJ Handover (Marketing Analytics)/Meridian MMM/MMM prog only data 2.csv"
df = pd.read_csv(fixed_path)

In [ ]:
coord_to_columns = load.CoordToColumns(
    time='week_start',
    geo=None,   # if you don’t have geo split, just leave None
    controls=[],  # e.g., competitor sales or macro controls if you have them
    population=None,  # if you don’t have population data
    kpi='event_registrants',
    # revenue_per_kpi=None,  # unless you can assign $ per registrant
    media=[
        'content_s',
        'display_s',
        'ooh_static_s',
        'search_s',
        'social_s',
        'video_s',
    ],
    media_spend=[
        'content_s',
        'display_s',
        'ooh_static_s',
        'search_s',
        'social_s',
        'video_s',
    ],
    organic_media=[
        'facebook_organic_impressions',
    ],
    non_media_treatments=[
        'public_holiday_flag',
        'school_holiday_flag',
    ],
)

In [ ]:
correct_media_to_channel = {
    "content_s": "Content",
    "display_s": "Display",
    "ooh_static_s": "OOH_Static",
    "search_s": "Search",
    "social_s": "Social",
    "video_s": "Video",
}

correct_media_spend_to_channel = {
    "content_s": "Content",
    "display_s": "Display",
    "ooh_static_s": "OOH_Static",
    "search_s": "Search",
    "social_s": "Social",
    "video_s": "Video",
}

correct_media_spend_to_channel.update(correct_media_to_channel)

In [ ]:
loader = load.CsvDataLoader(
    csv_path=fixed_path,
    kpi_type="non_revenue",
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)

data = loader.load()

/usr/local/lib/python3.12/dist-packages/meridian/data/input_data.py:524: UserWarning: Consider setting custom priors, as kpi_type was specified as `non_revenue` with no `revenue_per_kpi` being set. Otherwise, the total media contribution prior will be used with `p_mean=0.4` and `p_sd=0.2`. Further documentation available at https://developers.google.com/meridian/docs/advanced-modeling/unknown-revenue-kpi-custom#set-total-paid-media-contribution-prior
  warnings.warn(


In [ ]:
roi_mu = 0.2     # Mu for ROI prior for each media channel. (Mean)
roi_sigma = 0.9  # Sigma for ROI prior for each media channel. (Standard Deviation)
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(roi_mu, roi_sigma, name=constants.ROI_M)
)
model_spec = spec.ModelSpec(prior=prior)

mmm = model.Meridian(input_data=data, model_spec=model_spec)

/usr/local/lib/python3.12/dist-packages/meridian/model/context.py:694: UserWarning: Consider setting custom ROI priors, as kpi_type was specified as `non_revenue` with no `revenue_per_kpi` being set. Otherwise, the total media contribution prior will be used with `p_mean=0.4` and `p_sd=0.2`. Further documentation available at  https://developers.google.com/meridian/docs/advanced-modeling/unknown-revenue-kpi-custom#set-total-paid-media-contribution-prior
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/model.py:74: UserWarning: In a nationally aggregated model, the `media_effects_dist` will be reset to `normal`.
  warnings.warn(


In [ ]:
#%%time

mmm.sample_prior(500)
#It tells the function to generate 500 samples from the prior distribution.x



from tqdm.notebook import tqdm  # Import tqdm for Jupyter Notebook
import time

# ... (rest of your imports and code) ...

with tqdm(total=5*1000, desc="Training Progress") as pbar:  # Total iterations
    def update_progress(current_iteration, total_iterations):
        pbar.update(current_iteration)  # Update the progress bar

    # Remove progress_callback from the sample_posterior call
    mmm.sample_posterior(
        n_chains=5,
        n_adapt=500,
        n_burnin=500,
        n_keep=1000,
        parallel_iterations=100,
        # progress_callback=update_progress  # Remove this line
    )
    # Manually update progress bar after sampling
    pbar.update(5 * 1000)

#n_chains: Markov Chain Monte Carlo (MCMC) indipendent chains
#n_adapt: number of initial samples used to tune the sampling algorithm for better performance
#n_burnin: number of initial samples from each chain that are discarded
#n_keep: number of samples to keep from each chain after the burn-in phase. These samples represent the posterior distribution of the model parameters and are used for inference.

/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. tau_g_excl_baseline has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_m has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_rf has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/model/prior_distribution.py:1265: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_om has been automatically se

Training Progress:   0%|          | 0/5000 [00:00<?, ?it/s]

In [ ]:
model_diagnostics = visualizer.ModelDiagnostics(mmm)
model_diagnostics.plot_rhat_boxplot()

/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:944: UserWarning: Revenue analysis is not available when `revenue_per_kpi` is unknown. Defaulting to KPI analysis.
  warnings.warn(


alt.LayerChart(...)

In [ ]:
mmm_summarizer = summarizer.Summarizer(mmm)

#save output
from google.colab import drive
drive.mount('/content/drive')

filepath = '/content/drive/MyDrive/NLB Internship'
start_date = '2023-01-02'
end_date = '2024-12-30'
mmm_summarizer.output_model_results_summary('mmm_summary_output.html', filepath, start_date, end_date)
#preview 2 pager
IPython.display.HTML(filename='/content/drive/MyDrive/NLB Internship/mmm_summary_output.html')


/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:944: UserWarning: Revenue analysis is not available when `revenue_per_kpi` is unknown. Defaulting to KPI analysis.
  warnings.warn(


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:629: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:629: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/numpy/lib/_function_base_impl.py:4779: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
/usr/local/lib/python3.12/dist-packages/meridian/analysis/analyzer.py:3238: UserWarning: Effectiveness is not reported because it does not have a clear interpretation by time period.
  warnings.warn(


Dataset,R-squared,MAPE,wMAPE
All Data,0.15,24%,19%


## Optimize Budget

In [ ]:
%%time
budget_optimizer = optimizer.BudgetOptimizer(mmm)
optimization_results = budget_optimizer.optimize(use_kpi=True)

CPU times: user 27.2 s, sys: 682 ms, total: 27.9 s
Wall time: 19.5 s


In [ ]:
filepath = '/content/drive/MyDrive/NLB Internship'
optimization_results.output_optimization_summary('mmm_optimization_output.html', filepath)
IPython.display.HTML(filename='/content/drive/MyDrive/NLB Internship/mmm_optimization_output.html')

Channel,Non-optimized spend,Optimized spend
Social,51%,52%
Content,15%,16%
Search,15%,15%
OOH_Static,12%,8%
Display,4%,4%
Video,3%,4%
